In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from time import sleep

In [ ]:
spark = SparkSession.builder.appName('Basic streaming').getOrCreate()

In [ ]:
#Get schema of dataset 
static = spark.read.json("D:/Dataset/activity-data")
static.printSchema()
dataSchema = static.schema

In [ ]:
# Create Streming Dataset
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1)\
  .json("D:/Dataset/activity-data")

In [ ]:
# Get required info from dataset
activityCounts = streaming.groupBy("gt").count()

In [ ]:
# Specify the numner of shuffle partitions
spark.conf.set("spark.sql.shuffle.partitions", 1)

In [ ]:
# Generate query by specifying output sink using writeStream 
activityQuery = activityCounts.writeStream.queryName("activity_count")\
  .format("memory").outputMode("complete")\
  .start()

In [ ]:
# Any number of queries (usecases) can be created
#userCounts = streaming.groupBy("").count()
userCounts = streaming.groupBy('User').count()
userQuery = userCounts.writeStream.queryName("user_count")\
  .format("memory").outputMode("complete")\
  .start()

In [ ]:
# Display result
for x in range(5):
    spark.sql('SELECT * FROM activity_count').show()
    spark.sql('SELECT * FROM user_count').show()
    sleep(10)

In [ ]:
# Run nonstop
activityQuery.awaitTermination()